# ODIN & NN Softmax OOD detectors

This notebook represents the implementation of two OOD detectors which are ODIN and NN Softmax. 
First, we load and preprocess images from the ISIC2019 as In ditibution dataset and Fitzpatick17k as OOD dataset using the data generators.
Then we load the DensNet model that was trained on the ISIC2019 Dataset inorder to evaluate both ODIN and NN softmax baseline scores to get the optimal parameters and optimal delta.

#### Imports

In [ ]:
import tensorflow as tf
import tensorflow

from tensorflow import keras
from keras.layers import Dense

import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tqdm import trange

import keras.backend as K
import cv2
from odin import norm_perturbations
from scipy.special import softmax

from odin import get_tpr_and_fpr, auroc
from scipy.special import softmax
from sklearn.metrics import roc_curve, auc

import cv2



# 1. Load and preprocess the training and testing Datasets 

First we load and preprocess The training dataset ISIC2019, images are randomly cropped, resized, and possibly flipped to augment the data and improve the model's robustness. During validation, images are simply resized and normalized.

Then we load and preprocess the  testing dataset FitzPatrick17K, the fitzpatick categories are loaded separately in the testing generator , we have 3 categories: FST I-IV ,FST V-VI ,ALL Fitzpatrick images refering to the skin tones categories of the testing dataset.

In [ ]:
from argument_parser import myParser

args = myParser()

Training using the following parameters:
augment: True
batch_size: 40
checkpoint_frequency: 1000
dataset_root: /usr/xtmp/hannah/segkeras/nn-isic2019/dataset/
decay_start_iteration: 15000
detailed_logs: False
early_stop_patience: 8
experiment_root: FL_Aug_MEL
gpu_device: 0
heavy_augment: False
init_epoch: 0
initial_checkpoint: None
is_train: 1
learning_rate: 1e-06
load_weights: None
loading_threads: 8
loss_type: 1
margin: soft
num_classes: 7
num_epochs: 25
num_gpus: 1
resume: False
sizeH: 224
sizeW: 224
skip_class: MEL
train_iterations: 25000


In [ ]:
# The training dataset ISIC2019
from data_generators import ISICImageDataset

## Loading Data Generators
ROOT = "/ISIC_2019_Training_Input/"

print("Loading Data Generators...")
args.is_train = 2  # Training Set
args.num_classes = 6
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2  # Validation Set
Validation_generator = ISICImageDataset(args)
print("   Done.")

Loading Data Generators...
class_names:['MEL', 'NV', 'BCC', 'AK', 'BKL', 'SCC']
skip_class:['DF', 'VASC']
Loading Testing Dataset...
   Done.


In [ ]:
from data_generators import fitzpatrick17kImageDataset

## Loading Data Generators
ROOT = '/fitzpatrick 17k /'

print('Loading Data Generators...')

#Fitzpatick FST I-IV 
args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2 
Testing_generator_FST_V-VI = fitzpatrick17kImageDataset(args)

print('   Done.')


Loading Data Generators...
Loading Testing Dataset...
   Done.


In [ ]:
#Fitzpatick FST I-IV 
from data_generators import fitzpatrick17kImageDataset
## Loading Data Generators
ROOT = '/fitzpatrick 17k /'

print('Loading Data Generators...')

args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 2 # Validation Set  change the condition on common.py before running
Testing_generator_FST_I-IV = fitzpatrick17kImageDataset(args)

print('   Done.')



Loading Data Generators...
Loading Testing Dataset...
   Done.


In [ ]:
# Fitzpatick ALL images
from data_generators import fitzpatrick17kImageDataset

## Loading Data Generators
ROOT = "/fitzpatrick 17k /"

print("Loading Data Generators...")

args.num_classes = 3
args.sizeW = 224
args.sizeH = 224
args.dataset_root = ROOT
args.is_train = 0
Testing_generator_ALL = fitzpatrick17kImageDataset(args)

print("   Done.")

Loading Data Generators...
Loading Validation Dataset...
   Done.


# 2. Model Evaluation

#### 2.1 Load the pretrained DenseNet model 

In [ ]:
print("Loading Network...")
model_path = "../Models/my_model_densenet.h5"
densenet_model = tf.keras.models.load_model(model_path)
print(" Done.")

Loading Network...


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Colocations handled automatically by placer.


 Done.


In [ ]:

from ODIN_aux import compute_baseline_softmax_scores 
from ODIN_aux import get_perturbation_helper_func
from ODIN_aux import compute_odin_softmax_scores

# Find the best parameters :

In [ ]:
compute_odin_softmax_scores( densenet_model, Validation_generator, Testing_generator_FST_V - VI)

In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_I - IV)

In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_ALL)

In [ ]:
#testing the subset of parameters


need_norm_perturbations = 1
best_auroc = 0.0
best_auroc_params = None
lowest_fpr = float('inf')
lowest_fpr_params = None


#We want to loop over all the saved scores
In_scores_files ="/Models/ODIN/Saved scores/In"
Out_scores_files = "/Models/ODIN/Saved scores/Out"




in_dist_file="densenet121_ODIN_{}_{}_In.txt".format(temperature, magnitude)
out_dist_file="densenet121_ODIN_{}_{}_Out.txt".format(temperature, magnitude)



 # Calculate AUROC
auroc_value = auroc(in_dist_file=  , out_dist_file=out_dist_file)

                    # Calculate FPR
tpr, fpr = get_tpr_and_fpr(scores_in_test, scores_out_test, delta)

                    # Update best AUROC and FPR if necessary
if auroc_value > best_auroc:
   best_auroc = auroc_value
   best_auroc_params = (temperature, magnitude, dist)

if fpr < lowest_fpr:
   lowest_fpr = fpr
   lowest_fpr_params = (temperature, magnitude, dist)

print("Best AUROC: {:.4f}, Parameters: Temperature={}, Magnitude={}, Distribution={}"
      .format(best_auroc, *best_auroc_params))

    # Print the lowest FPR and corresponding parameters
print("Lowest FPR: {:.4f}, Parameters: Temperature={}, Magnitude={}, Distribution={}"
      .format(lowest_fpr, *lowest_fpr_params))

In [ ]:
compute_odin_softmax_scores(
    densenet_model, Validation_generator, Testing_generator_FST_V - VI
)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax:0", shape=(1, ?), dtype=int64)
Skip  In, 1000, 0.0

===== Temperature: 1000, Magnitude: 0.0, Out-Distribution =====


100%|██████████| 1318/1318 [27:31<00:00,  1.25s/it]



===== Temperature: 1000, Magnitude: 0.0002, In-Distribution =====


100%|██████████| 2495/2495 [29:22<00:00,  1.42it/s]



===== Temperature: 1000, Magnitude: 0.0002, Out-Distribution =====


...

100%|██████████| 2495/2495 [30:12<00:00,  1.38it/s]



===== Temperature: 1000, Magnitude: 0.0014, Out-Distribution =====


100%|██████████| 1318/1318 [15:34<00:00,  1.41it/s]



===== Temperature: 1000, Magnitude: 0.0016, In-Distribution =====


 41%|████      | 1013/2495 [12:01<18:12,  1.36it/s]

In [ ]:
compute_odin_softmax_scores(densenet_model, Validation_generator, Testing_generator_FST_I - IV)

## Setting the optimal parameters :

In [ ]:
from ODIN_aux import compute_odin_parameters_and_metrics
from ODIN_aux import compute_odin_softmax_scores_fixed


In [ ]:


# Optimal ODIN Parameters
optimal_temperature = 200
optimal_magnitude = 0.0002
optimal_delta = 0.90385

# Fixed ODIN Parameters
# optimal_temperature = 2
# optimal_magnitude = 0.0002

# Call the function to compute ODIN parameters and metrics
# compute_odin_parameters_and_metrics(densenet_model, Validation_generator, Testing_generator_lighter_only)

## ODIN scores

In [ ]:
compute_odin_softmax_scores_fixed(densenet_model, Validation_generator, Testing_generator_FST_V - VI)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv_6:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax_12:0", shape=(1, ?), dtype=int64)

===== Temperature: 200, Magnitude: 0.0002, Out-Distribution =====


100%|██████████| 1318/1318 [15:24<00:00,  1.43it/s]


In [ ]:
compute_odin_softmax_scores_fixed(densenet_model, Validation_generator, Testing_generator_FST_I - IV)

In [ ]:
compute_odin_softmax_scores_fixed(densenet_model, Validation_generator, Testing_generator_ALL)

Tensor("dense_4/BiasAdd:0", shape=(?, 6), dtype=float32)
Tensor("truediv_2:0", shape=(?, 6), dtype=float32)
Tensor("ArgMax_4:0", shape=(1, ?), dtype=int64)

===== Temperature: 200, Magnitude: 0.0002, Out-Distribution =====


100%|██████████| 12222/12222 [2:11:44<00:00,  1.55it/s]


## Baseline scores

In [ ]:
# lFST I-IV
compute_baseline_softmax_scores(
    densenet_model, Validation_generator, Testing_generator_FST_I - IV
)

Begin to compute baseline softmax scores


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
# FST V-VI
compute_baseline_softmax_scores(
    densenet_model, Validation_generator, Testing_generator_FST_V - VI
)

Begin to compute baseline softmax scores


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2359: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


In [ ]:
# ALL
compute_baseline_softmax_scores(
    densenet_model, Validation_generator, Testing_generator_ALL
)

Begin to compute baseline softmax scores


In [ ]:
# find the ptimal threshold:
# Concatenate the scores and create corresponding labels

scores_in = np.loadtxt("/content/densenet121_ODIN_200_0.0002_In_FST_V_VI.txt")
scores_out = np.loadtxt("/content/densenet121_ODIN_200_0.0002_Out_FST_V_VI.txt")
scores_D = np.concatenate([scores_in, scores_out])
labels_D = np.concatenate([np.zeros(len(scores_in)), np.ones(len(scores_out))])

# Use ROC curve to find the optimal threshold
fpr, tpr, thresholds = roc_curve(labels_D, scores_D)
optimal_threshold_D = thresholds[np.argmax(tpr - fpr)]

In [ ]:
print(optimal_threshold)  # ODIN

0.17385952174663544


In [ ]:
print(optimal_threshold)  # NN Softmax

0.9962658286094666
